In [270]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/titanic.csv


In [271]:
dataset=pd.read_csv("/kaggle/input/titanic/titanic.csv")
train=dataset.copy()
test=dataset.copy()
train.drop(['PassengerId','Fare','Cabin','Name'],axis=1,inplace=True)
train.drop(['Ticket'],axis=1,inplace=True)
train['Age'].fillna(28, inplace=True)
train['Embarked'].fillna('S', inplace=True)
train['TravelAlone']=np.where((train["SibSp"]+train["Parch"])>0, 0, 1)
train.drop('SibSp', axis=1, inplace=True)
train.drop('Parch', axis=1, inplace=True)
train['Sex'] = train['Sex'].replace(['male','female'],[0,1]).astype(int)
train['Embarked'] = train['Embarked'].replace(['S','Q','C'],[1,2,3]).astype(int)
train['Pclass'] = train['Embarked'].replace(['1','2','3'],[3,2,1]).astype(int)
test.drop(['PassengerId','Fare','Cabin','Name','Ticket'],axis=1,inplace=True)
test['Age'].fillna(28, inplace=True)
test['Embarked'].fillna('S', inplace=True)


test['TravelAlone']=np.where((test["SibSp"]+test["Parch"])>0, 0, 1)

test.drop('SibSp', axis=1, inplace=True)
test.drop('Parch', axis=1, inplace=True)

test['Sex'] = test['Sex'].replace(['male','female'],[0,1]).astype(int)
test['Embarked'] = test['Embarked'].replace(['S','Q','C'],[1,2,3]).astype(int)
test['Pclass'] = test['Embarked'].replace(['1','2','3'],[3,2,1]).astype(int)
train['IsMinor']=np.where(train['Age']<=16, 1, 0)

test['IsMinor']=np.where(test['Age']<=16, 1, 0)
train



,Survived,Pclass,Sex,Age,Embarked,TravelAlone,IsMinor
0,0,1,0,22.0,1,0,0
1,1,3,1,38.0,3,0,0
2,1,1,1,26.0,1,1,0
3,1,1,1,35.0,1,0,0
4,0,1,0,35.0,1,1,0
...,...,...,...,...,...,...,...
886,0,1,0,27.0,1,1,0
887,1,1,1,19.0,1,1,0
888,0,1,1,28.0,1,0,0
889,1,3,0,26.0,3,1,0


In [272]:
y=train['Survived']
X = train[['Pclass', 'Sex', 'Age', 'Embarked','TravelAlone','IsMinor']]
for i in X.T:
    fmin=X.min()
    frange=X.max()-X.min()
    X-=fmin
    X/=frange
X = np.asarray(X, dtype='float64')
y = np.asarray(y, dtype='float64')

In [273]:
#training=train.copy()
#testing=test.copy()
#training.drop('Survived', axis=1, inplace=True)
#testing.drop('Survived', axis=1, inplace=True)
#X_train = np.array(training).T
#Y_train = np.array(Y_train)
#Y_train.shape

In [274]:
#Y_train = Y_train.reshape(Y_train.shape[0], 1).T
#Y_test = np.array(Y_test)
#Y_test = Y_test.reshape(Y_test.shape[0], 1).T
#X_test = np.array(testing).T

#assert X_train.shape[1] == Y_train.shape[1]
#assert X_train.shape[0] == X_test.shape[0]
#X_train.shape, Y_train.shape,Y_test.shape, X_test.shape

In [275]:
class SVM:

    def __init__(self, learning_rate=0.01, lambda_param=0.01, n_iters=4000):
        self.lr = learning_rate
        self.lambda_param = lambda_param
        self.n_iters = n_iters
        self.w = None
        self.b = None

    def fit(self, X, y):
        n_samples, n_features = X.shape

        y_ = np.where(y <= 0, -1, 1)

        # init weights
        self.w = np.zeros(n_features)
        self.b = 0

        for _ in range(self.n_iters):
            for idx, x_i in enumerate(X):
                condition = (y_[idx] * (np.dot(x_i, self.w) - self.b) >= 1).all()
                if condition:
                    self.w -= self.lr * (2 * self.lambda_param * self.w)
                else:
                    self.w -= self.lr * (2 * self.lambda_param * self.w - x_i * y_[idx])
                    self.b -= self.lr * y_[idx]


    def predict(self, X):
        print("Shapes:", X.shape, self.w.shape, self.b.shape)
        approx = np.dot(X, self.w) - self.b
        return np.sign(approx)

In [276]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)
clf = SVM()
clf.fit(x_train, y_train)

predictions = clf.predict(x_test)
def accuracy(y_true, y_pred):
    accuracy = np.sum(y_true == y_pred) / len(y_true)
    return accuracy
print("SVM classification accuracy", accuracy(y_test, predictions))

Shapes: (179, 6) (6,) ()
SVM classification accuracy 0.2569832402234637
